In [2]:
import numpy as np
import scipy as sp
import nuSQUIDSpy as nsq

In [32]:
units = nsq.Const()
gr = nsq.GlashowResonanceCrossSection()
dis = nsq.NeutrinoDISCrossSectionsFromTables()

In [33]:
def TotalNeutrinoCrossSection(enu, 
                              flavor = nsq.NeutrinoCrossSections_NeutrinoFlavor.electron,
                              neutype = nsq.NeutrinoCrossSections_NeutrinoType.neutrino):
    return np.sum([dis.TotalCrossSection(enu*units.GeV,flavor,neutype,interaction) for interaction in
            [nsq.NeutrinoCrossSections_Current.CC,nsq.NeutrinoCrossSections_Current.NC]])

def DifferentialOutGoingLeptonDistribution(enu_in,enu_out,
                                       flavor = nsq.NeutrinoCrossSections_NeutrinoFlavor.tau,
                                       neutype = nsq.NeutrinoCrossSections_NeutrinoType.neutrino,
                                       interaction = nsq.NeutrinoCrossSections_Current.NC
                                    ):
    total = dis.TotalCrossSection(enu_in*units.GeV,flavor,neutype,interaction)
    diff = dis.SingleDifferentialCrossSection(enu_in*units.GeV,enu_out*units.GeV,flavor,neutype,interaction)
    return diff/total

def DistributionOfTauDecays(tau_initial,tau_final):
    #make me
    return 1.

In [34]:
def InteractionLength(xs, density):
    return 1./(xs*density)

In [30]:
LifeTimeDictionary = {"tau_neutrino": np.inf, "tau": 1.*units.sec};
MassDictionary = {"tau_neutrino": 0., "tau": 1.7*units.GeV};
proton_mass = 1.*units.GeV

class CasinoEvent(object):
    def __init__(self, particle_id, energy, position):
        self.particle_id = particle_id
        self.energy = energy
        self.position = position
    def GetDecayProbability(self,dL):
        # check me
        boost_factor = (self.energy/MassDictionary[self.particle_id])
        return dL*boost_factor/LifeTimeDictionary[self.particle_id]
    def GetInteractionProbability(self,dL,density):
        # check me
        return dL/(TotalNeutrinoCrossSection(self.energy)*density/proton_mass)

In [31]:
TotalDistance = 100*units.km
ProposedDistanceStep = 1.*units.km 
density = 1.*units.gr/units.cm**3

In [26]:
FirstEvent = CasinoEvent("tau_neutrino",1e7*units.GeV,0.0)

In [ ]:
EventCollection = [FirstEvent]

while(map(lambda e: e.position < TotalDistance,EventCollection)):
    for(event in EventCollection):
        if(event.GetDecayProbability(ProposedDistanceStep)<p):
            # do decay
            event.particle_id = "tau_neutrino"
            event.energy = Sample(lambda enu_out: DistributionOfTauDecays(e.energy, enu_out))
        if(event.GetInteractionProbability(ProposedDistanceStep,density)<p):
            # do interaction
            if event.particle_id = "tau_neutrino":
                event.energy = Sample(lambda enu_out: DifferentialOutGoingNeutrinoLeptonDistribution(e.energy, enu_out))
            else if event.particle_id = "tau":
                event.energy = Sample(lambda enu_out: DifferentialOutGoingTauLeptonDistribution(e.energy, enu_out))
    # move everything forward
    for(event in EventCollection):
        event.position += ProposedDistanceStep
